In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

traning the model using LGBM along with cross validation

import all the data required here I already created folds out of data in my previous notebook so just importing that data into df_train

https://www.kaggle.com/pavansaikondalarao/creating-kfolds-tabular-playground-aug-2021

In [ ]:
df_train = pd.read_csv("../input/tabulardata-kfolds-created/train_folds.csv")
df_test = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")
sample_submission = pd.read_csv("../input/tabular-playground-series-aug-2021/sample_submission.csv")

In [ ]:
sample_submission.head()

we need to exclude certain columns before feeding our data into the model.

In [ ]:
useful_features = [c for c in df_train.columns if c not in ('id','loss','kfold')]
df_train[useful_features]

performing LGBM on all the 5 datasets we created using kfolds and storing the predicted values in a list called **final_preds**

In [ ]:
from lightgbm import LGBMRegressor
import numpy as np
final_preds=[]
for fold in range (5):
    xtrain = df_train[df_train.kfold != fold]
    xvalid = df_train[df_train.kfold == fold]
    ytrain = xtrain['loss']
    xtrain = xtrain[useful_features]
    yvalid = xvalid['loss']
    xvalid = xvalid[useful_features]
    model = LGBMRegressor()
    model.fit(xtrain,ytrain)
    preds_valid = model.predict(xvalid)
    test = df_test[useful_features]
    test_preds = model.predict(test)
    final_preds.append(test_preds)
    print(mean_squared_error(yvalid,preds_valid,squared=False))

we got the predictions from and we now average them to get our final predicted values for test set 

In [ ]:
preds = np.mean(np.column_stack(final_preds),axis=1)

In [ ]:

sample_submission['loss'] = preds
sample_submission.to_csv('output.csv',index=False)
